In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


"""Change to the data folder"""
new_path = "../new_train/"
val_path = "../new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, training=True):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.training = training

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)

#         varMin =[-5.37872696e+00,  0.00000000e+00, -6.39434204e+01, -8.02092514e+01,
#   0.00000000e+00,  0.00000000e+00,  1.00000000e+06,  1.00000000e+06]
#         varMax = [ 4.72452881e+03,  4.08314209e+03,  7.73842545e+01,  0.00000000e+00,
#   4.73215820e+03,  4.07677856e+03, -1.00000000e+06, -1.00000000e+06]
        columns = ['p_in','p_in','v_in','v_in','lane','lane','lane_norm','lane_norm']
        outColumns = ['p_out','p_out','v_out','v_out']
    #FOR 1000 rows, no shuffle
#     varMin=[ 0.00000000e+00,  0.00000000e+00, -3.83204346e+01, -4.79440918e+01,
#   0.00000000e+00,  0.00000000e+00,  1.00000000e+06,  1.00000000e+06]
#         varMax=[ 4.70824121e+03,  4.04640869e+03,  7.00706635e+01,  0.00000000e+00,
#   4.71727344e+03,  4.07275757e+03, -1.00000000e+06, -1.00000000e+06]
#   FOR ALL ROWS
        varMaxOutput= [4773.,   4097.7,   193.19,  194.33]
        varMinOutput=[ -53.912,    0.,    -210.04,  -187.71 ]
        varMaxInput=[4748.2,   4096.1,    252.32,   183.53,  4791.6,   4121.4,     18.801,   16.702]
        varMinInput=[ -46.958,    0.,    -222.63,  -179.87,   -75.963,    0.,     -18.564,  -16.691]
#         #Changed to 4 because we don't need lane and lane norm
        for i in range(8):
            j = i % 2
            data[columns[i]][j] = (data[columns[i]][j] - varMinInput[i]) / (varMaxInput[i] - varMinInput[i])
            if i < 4 and self.training:
                data[outColumns[i]][j] = (data[outColumns[i]][j] - varMinOutput[i]) / (varMaxOutput[i] - varMinOutput[i]) 
#         data['p_out'][0] = (data['p_out'][0] - -5.37872696e+00) / (4.72452881e+03 - -5.37872696e+00)
        return data


train_dataset  = ArgoverseDataset(data_path=new_path)
val_dataset = ArgoverseDataset(data_path=val_path,training=False)
#print((val_dataset[0]))
#print(len(train_dataset[0]))

### Create a loader to enable batch processing

In [3]:
batch_sz = 200

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = []
    out = []
    laneInfo = []
    city = []
#     print("pIn",batch[0]['p_in'])
    numbRows = 60
    for scene in batch:
#         print("scenePin",scene['p_in'])
        cityyy = numpy.zeros((60,1,4))
        #print(cityyy)
        if scene['city'] == 'PIT':
            cityyy[:,:,:] = 1
            #print("mummamia",cityyy)
        city.append(numpy.dstack([cityyy]))
        lanes = numpy.zeros((numbRows * 19,2))
        lane_norm = numpy.zeros((numbRows * 19,2))
#         pIn = numpy.zeros((numbRows,19,2))
#         vIn = numpy.zeros((numbRows,19,2))
        lengthLane = min(numbRows * 19,len(scene['lane']))
#         pIn[:len(scene['p_in']),:,:2] = scene['p_in']
#         vIn[:len(scene['v_in']),:,:2] = scene['v_in']
        
#         x = lanes.reshape()
        lanes[:lengthLane,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,:2] = scene['lane_norm'] [:lengthLane,:2]
        laneInfo.append(numpy.dstack([lanes.reshape(60,19,2),lane_norm.reshape(60,19,2)]))
        inp.append(numpy.dstack([scene['p_in'],scene['v_in']]))
        out.append(numpy.dstack([scene['p_out'], scene['v_out']]))
#     print("from mycollate",scene['p_in']) 
#     print("p_in",inp[0][0][0])
    laneInfo = torch.FloatTensor(laneInfo)
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    city = torch.FloatTensor(city)
    return [city,laneInfo,inp, out]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=0)

In [4]:

def val_collate(batch):
    agentIds = []
    trackIds = []
    sceneIdxs = []
    laneInfo = []
    inp = []
    for scene in batch:
        """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
        agentIds.append(scene['agent_id'])
        trackIds.append(scene['track_id'])
        sceneIdxs.append(scene['scene_idx'])
        lanes = numpy.zeros((numbRows * 19,2))
        lane_norm = numpy.zeros((numbRows * 19,2))
        lengthLane = min(numbRows * 19,len(scene['lane']))
        lanes[:lengthLane,:2] = scene['lane'][:lengthLane,:2]
        lane_norm[:lengthLane,:2] = scene['lane_norm'] [:lengthLane,:2]
        laneInfo.append(numpy.dstack([lanes.reshape(60,19,2),lane_norm.reshape(60,19,2)]))
        inp.append(numpy.dstack([scene['p_in'], scene['v_in']]))
    inp = torch.FloatTensor(inp)
    laneInfo = torch.FloatTensor(laneInfo)
    return [inp,sceneIdxs,agentIds,trackIds,laneInfo]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()
        

        self.hidden_dim = hidden_dim
        self.num_layers = n_layers
        
        self.lstm = nn.LSTM(input_size, self.hidden_dim, self.num_layers, batch_first=True,dropout=0.5)
        self.fc = nn.Conv1d(self.hidden_dim, 240,1)
#         self.bn1 = nn.BatchNorm1d(num_features=4)
        
    def forward(self, x):
        x,_ = self.lstm(x)
#         print("shapeee",x.shape)
        x = x.transpose(1,2)
#         print(x.shape)
        x = self.fc(x)
#         print(x.shape)
        x = x.transpose(1,2)

        return x
    
    def forwardTesting(self,x):
        res =[]
        h = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        c = torch.zeros((self.num_layers,len(x),self.hidden_dim)).cuda()
        for steps in range(num_steps):
            x,(h,c) = self.lstm(x,(h,c))
            x = x[:,-1:]
            x = x.tranpose(1,2)
            x = self.fc(x)
            x = x.transpose(1,2)
            res.append(x)


In [6]:
def make_a_histogram(sample_batch, agent_id, xPos, yPos, xVel, yVel):
    inp, out = sample_batch
    batch_sz = inp.size(0)
    #agent_sz = inp.size(1)
    
    for i in range(batch_sz):
        #hist_data_xPos = np.zeros((60,19));
        #hist_data_yPos = np.zeros((60,19));
        #hist_data_xVel = np.zeros((60,19));
        hist_data_yVel = np.zeros((60,19));
        
        for j in range(60):
            #hist_data_xPos[j] = (inp[i, j,:,0])
            #hist_data_yPos[j] = (inp[i, j,:,1])
            #hist_data_xVel[j] = (inp[i, j,:,2])
            hist_data_yVel[j] = (inp[i, j,:,3])
            
        for j in range(len(hist_data_yVel)):
            for k in range(len(hist_data_yVel[j])):
                #xPos.append(hist_data_xPos[j][k])
                #yPos.append(hist_data_yPos[j][k])
                #xVel.append(hist_data_xVel[j][k])
                yVel.append(hist_data_yVel[j][k])
    
    """
    hist_data_xPos = np.zeros((60,19));
    hist_data_yPos = np.zeros((60,19));
    hist_data_xVel = np.zeros((60,19));
    hist_data_yVel = np.zeros((60,19));
    
    for i in range(60):
        hist_data_xPos[i] = (inp[0, i,:,0])
        hist_data_yPos[i] = (inp[0, i,:,1])
        hist_data_xVel[i] = (inp[0, i,:,2])
        hist_data_yVel[i] = (inp[0, i,:,3])
    
    xPos = np.zeros(60*19)
    for i in range(len(hist_data_xPos)):
        for j in range(len(hist_data_xPos[i])):
            xPos[i*19+j] = hist_data_xPos[i][j]
    
    #hist_data_xPos = hist_data_xPos.flatten()
    hist_data_yVel = hist_data_yPos.flatten()
    hist_data_xPos = hist_data_xVel.flatten()
    hist_data_yVel = hist_data_yVel.flatten()
    #print(xPos)
    
    n,bins,patches = plt.hist(x=xPos,bins='auto',alpha=0.7,rwidth=0.85)
    plt.grid(axis='y',alpha=0.75)
    maxfreq = n.max()
    plt.ylim(ymax=np.ceil(maxfreq/10) * 10 if maxfreq % 10 else maxfreq + 10)
    """

In [7]:
def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

### Visualize the batch of sequences

In [ ]:
from statistics import mean
import random
import numpy as np

torch.cuda.empty_cache()
agent_id = 0
learning_rate = 1e-3
momentum = 0.01
device = torch.device("cuda:0")
input_dim = 4 * 60    # input dimension
hidden_dim = 3000  # hidden layer dimension should be greater when batch_sz small
layer_dim = 3    # number of hidden layers
output_dim = 4   # output dimension

n_epochs = 30
# varMin =[-5.37872696e+00,  0.00000000e+00, -6.39434204e+01, -8.02092514e+01,
#   0.00000000e+00,  0.00000000e+00,  1.00000000e+06,  1.00000000e+06]
# varMax = [ 4.72452881e+03,  4.08314209e+03,  7.73842545e+01,  0.00000000e+00,
#   4.73215820e+03,  4.07677856e+03, -1.00000000e+06, -1.00000000e+06]
# columns = ['p_in','p_in','v_in','v_in','lane','lane','lane_norm','lane_norm']

# Define Loss, Optimizer
#model = RNNModel(input_dim, output_dim, hidden_dim, layer_dim).to(device)
model = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
model.load_state_dict(torch.load('./sliding3epoch.pth'))
model = model.to(device)
# optimizer = optim.Adagrad(model.parameters(), lr=learning_rate,weight_decay=0.00000001)
# optimizer = optim.Adadelta(model.parameters())
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)
# optimizer = optim.RMSprop(model.parameters(),lr=learning_rate,momentum=momentum)

#find mean
var = numpy.zeros((1,8))
seen = 0
print('start')

    
model.train()

print("test")
# newOut = torch.zeros((batch_sz,100,30,4))
batch = []
inpArray = []
loss_ema = -1
try:
    for i_epoch in range(n_epochs):
        for i_batch, sample_batch in enumerate(train_loader):
        #     print("test")
            city,laneInfo,inp, out = sample_batch
            laneInfo = laneInfo.cuda()
            inp = inp.cuda()
            out = out.cuda()
            city = city.cuda()
            optimizer.zero_grad()
            mixed = torch.cat([city,laneInfo,inp,out],2).transpose(1,2).reshape(-1,69,4 * 60)
#             print(mixed)
            
            y_pred = model(mixed[:,:-1])[:,-30:]
            y_pred = y_pred.reshape((-1,30,60,4)).transpose(1,2)
#             print(y_pred.shape)
            
#             print(out.shape)
            #loss = nn.MSELoss()
            #loss = loss(y_pred,out)
#             print(y_pred)
            loss = (torch.mean((y_pred-out)**2))**0.5
            loss.backward()
            optimizer.step()
            if loss_ema < 0:
                loss_ema = loss
            loss_ema= loss_ema*0.90 +loss*0.1

            if i_batch % 10 == 0:
                print("batch #: ",i_batch * batch_sz ," avg loss per scene(past 100): ",loss_ema.item(),loss.item())
                batch = []
                
except KeyboardInterrupt:
    print("savedModel")
    torch.save(model.state_dict(), './sliding3epoch.pth')


start
test
batch #:  0  avg loss per scene(past 100):  342.82025146484375 342.8202819824219


In [11]:
torch.save(model.state_dict(), './sliding3epoch.pth')

In [ ]:
import numpy as np
# import pandas as pd

save_file = "submissionsliding.csv"


header = ["ID"]
header += ["v"+str(x) for x in range(1, 61)]
print(header)

with open(save_file, 'w') as f:
    f.write(",".join(header)+"\n")

device = "cuda:0"

testmodel = RNNModel(input_size=input_dim, output_size=output_dim, hidden_dim=hidden_dim, n_layers=layer_dim)
testmodel.load_state_dict(torch.load('sliding3epoch.pth'))
testmodel.to(device)
testmodel.eval()

full_out = []
print("test")

print(bool_loc)
for i_batch, sample_batch in enumerate(val_loader):
    if i_batch % 10 == 0:
        print("batch #: ", i_batch)
#     print(len(sample_batch[1]))
#     print(sample_batch[0].shape)
    inp, scene_idx, agent_ids, track_ids,laneInfo = sample_batch
#     print(agent_id, track_id)
    bool_loc = np.stack([np.squeeze(track_id) == np.repeat(ag_id,60) for ag_id, track_id in zip(agent_ids, track_ids)])
    mixed = torch.cat([laneInfo,inp],2).transpose(1,2).reshape(-1,38,4 * 60)
    y_pred = model.forwardTesting(mixed.reshape(len(mixed),-1)).reshape((-1,60,30,4))
    
    out = y_pred[bool_loc]
#     print(type(out))
#     print("out shape after: ", out.shape)
#     full_out.append(out)
#     print(type(scene_idx))
#     print(",".join(out.detach().cpu().numpy().astype(int).astype(str)[0]))
#     fin_out = np.concatenate((scene_idx.reshape(-1,1), out.detach().cpu().numpy().astype(int)), axis=1)
    with open(save_file, "a") as f:
        for row, scene_num in zip(out.detach().cpu().numpy().astype(int).astype(str), scene_idx):
            f.write(str(scene_num)+","+",".join(row)+"\n")
#         np.savetxt(f, fin_out, fmt="%cd",delimiter=",", newline="\n")
